In [3]:
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split


In [490]:
import pickle
cleaned_data = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\cleaned_data_Maor.pkl")
X_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_train.pkl")
X_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_test.pkl")
y_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_train.pkl")
y_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_test.pkl")
predictions = pd.read_csv(r'C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\predictions\predictions.csv')

In [5]:
raw_data = pd.read_csv(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\raw\train_dataset_full.csv")

In [22]:
cleaned_data.shape

(365798, 14)

In [27]:
#find how many duplicated rows we have
duplicates = raw_data[raw_data.duplicated()]
duplicates.shape

(19510, 15)

369653

In [30]:
raw_data.head()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click
0,98528.0,2017-07-04 16:42,7716.0,C,405490.0,60305.0,3.0,NaN,3.0,Male,3.0,3.0,NaN,1.0,1.0
1,589714.0,2017-07-07 07:40,1035283.0,I,118601.0,28529.0,4.0,82527.0,10.0,Female,4.0,3.0,3.0,1.0,0.0
2,478652.0,2017-07-07 20:42,65994.0,H,359520.0,13787.0,4.0,NaN,4.0,Male,4.0,3.0,2.0,0.0,0.0
3,34536.0,2017-07-05 15:05,75976.0,H,405490.0,60305.0,3.0,NaN,3.0,Male,3.0,3.0,3.0,0.0,0.0
4,71863.0,2017-07-06 20:11,987498.0,C,405490.0,60305.0,3.0,NaN,2.0,Male,2.0,3.0,2.0,0.0,0.0


In [6]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365798 entries, 0 to 370631
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   session_id              365107 non-null  float64       
 1   DateTime                365073 non-null  datetime64[ns]
 2   user_id                 365798 non-null  float64       
 3   product                 365736 non-null  category      
 4   campaign_id             365072 non-null  category      
 5   webpage_id              365070 non-null  category      
 6   user_group_id           350779 non-null  category      
 7   gender                  351429 non-null  category      
 8   age_level               351436 non-null  float64       
 9   user_depth              350771 non-null  float64       
 10  city_development_index  266504 non-null  float64       
 11  var_1                   365737 non-null  float64       
 12  is_click                365798 non-

In [483]:
X_train['month'] = X_train['DateTime'].dt.month
X_train['day'] = X_train['DateTime'].dt.day
X_train['hour'] = X_train['DateTime'].dt.hour

X_test['month'] = X_test['DateTime'].dt.month
X_test['day'] = X_test['DateTime'].dt.day
X_test['hour'] = X_test['DateTime'].dt.hour

In [484]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    """
    Fills missing values in specified columns with their mode.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        columns (list): List of column names to process.

    Returns:
        pd.DataFrame: DataFrame with missing values imputed.
    """
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df


In [491]:
#Fill each Na with X_trai
columns_to_fill = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "age_level", "user_depth", "city_development_index", "var_1", "product_category",
                     "month", "day", "hour"]

X_train_filled = fill_missing_with_mode(X_train, columns_to_fill)
X_test_filled = fill_missing_with_mode(X_test, columns_to_fill)

In [493]:
X_train_filled.isna().sum()

session_id                562
DateTime                  589
user_id                     0
product                     0
campaign_id                 0
webpage_id                  0
user_group_id               0
gender                      0
age_level                   0
user_depth                  0
city_development_index      0
var_1                       0
product_category            0
dtype: int64

In [494]:
X_train_filled.drop(columns = ['session_id', 'DateTime'], inplace = True)
X_test_filled.drop(columns = ['session_id', 'DateTime'], inplace = True)

In [495]:
X_train_filled.isna().sum()

user_id                   0
product                   0
campaign_id               0
webpage_id                0
user_group_id             0
gender                    0
age_level                 0
user_depth                0
city_development_index    0
var_1                     0
product_category          0
dtype: int64

In [496]:
columns_to_d = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "product_category"]

X_train_filled_d = pd.get_dummies(X_train_filled, columns = columns_to_d)
X_test_filled_d = pd.get_dummies(X_test_filled, columns = columns_to_d)

In [497]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_filled_d, y_train)
y_pred = rf.predict(X_test_filled_d)

f1_score(y_test, y_pred)

KeyboardInterrupt: 

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_filled_d, y_train)

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_smote, y_train_smote)

# Predict and evaluate
y_pred2 = rf.predict(X_test_filled_d)



In [11]:
f1 = f1_score(y_test, y_pred2)
print(f"F1-score (minority class): {f1}")

F1-score (minority class): 0.08470909711943528


In [13]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93     68166
         1.0       0.09      0.09      0.09      4994

    accuracy                           0.87     73160
   macro avg       0.51      0.51      0.51     73160
weighted avg       0.88      0.87      0.87     73160



In [488]:
X_test_filled_d.isna().sum().sum()

816

In [498]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
#adjuct the balance of the classes
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42, solver='liblinear',
                           penalty='l2', C=0.1)
model.fit(X_train_filled_d, y_train)

y_pred_LR = model.predict(X_test_filled_d)
print(f1_score(y_test, y_pred_LR))

0.1453661769579145


In [499]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB
from sklearn.metrics import f1_score, classification_report
model = ComplementNB()
model.fit(X_train_filled_d, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_filled_d)
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")
print(classification_report(y_test, y_pred))

F1-score: 0.14614835099416598
              precision    recall  f1-score   support

         0.0       0.94      0.62      0.75     68166
         1.0       0.09      0.49      0.15      4994

    accuracy                           0.61     73160
   macro avg       0.51      0.55      0.45     73160
weighted avg       0.88      0.61      0.70     73160



In [19]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42, class_weight='balanced')
model.fit(X_train_filled_d, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_filled_d)
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")

F1-score: 0.09819028847876725


In [15]:
from sklearn.metrics import classification_report  
print(classification_report(y_test, y_pred_LR))

              precision    recall  f1-score   support

         0.0       0.94      0.57      0.71     68166
         1.0       0.08      0.54      0.15      4994

    accuracy                           0.57     73160
   macro avg       0.51      0.55      0.43     73160
weighted avg       0.89      0.57      0.67     73160



**Baseline? :()**

In [16]:
cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

0.9323260378678943

In [396]:
a = 1- cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

In [6]:
X_test.head()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,user_group_id,gender,age_level,user_depth,city_development_index,var_1,product_category
370001,392485.0,2017-07-02 11:17:00,308642.0,D,82320,1734,12,Female,6.0,3.0,4.0,0.0,1
227178,63873.0,2017-07-03 16:25:00,238993.0,C,405490,60305,3,Male,3.0,3.0,2.0,1.0,3
84936,475022.0,2017-07-02 09:46:00,3307.0,H,359520,13787,4,Male,4.0,3.0,2.0,0.0,4
110154,524314.0,2017-07-04 19:25:00,946940.0,H,105960,11085,3,Male,3.0,2.0,NaN,1.0,4
6988,3653.0,2017-07-02 12:13:00,860243.0,I,404347,53587,9,Female,3.0,3.0,3.0,0.0,1


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
                random_seed=42, verbose=0, eval_metric='F1',
                cat_features=cat_features, class_weights=[1, 10]
            )
model.fit(X_train_filled_d, y_train)



In [74]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292638 entries, 196 to 123594
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   session_id              292076 non-null  float64       
 1   DateTime                292049 non-null  datetime64[ns]
 2   user_id                 292638 non-null  float64       
 3   product                 292638 non-null  category      
 4   campaign_id             292638 non-null  category      
 5   webpage_id              292638 non-null  category      
 6   user_group_id           292638 non-null  category      
 7   gender                  292638 non-null  category      
 8   age_level               281178 non-null  float64       
 9   user_depth              280646 non-null  float64       
 10  city_development_index  213253 non-null  float64       
 11  var_1                   292591 non-null  float64       
 12  product_category        292638 no

In [423]:
X_train['hour'] = X_train['DateTime'].dt.hour
X_train['day'] = X_train['DateTime'].dt.day
X_train['month'] = X_train['DateTime'].dt.month

In [424]:
X_train1 = X_train.drop(columns=['session_id', 'DateTime', 'user_id'])

In [425]:
def objective(trial):
    # --- 1) Suggest hyperparams ---
    # Example search space: Feel free to expand or tune ranges
    params = {
        "iterations": 1000,
        "depth": trial.suggest_int("depth", 4, 8),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        # Commonly used knobs; random_strength, subsample, etc. can also be tuned
        "eval_metric": "F1",
        "random_seed": 42,
        "auto_class_weights": "Balanced",
        "verbose": 0  # Keep CatBoost silent
    }

    # --- 2) Create CatBoost model ---
    model = CatBoostClassifier(**params)

    # --- 3) Train/Validation split ---
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
        X_train1,
        y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train
    )

    # --- 4) Train ---
    model.fit(
        X_train_sub,
        y_train_sub,
        cat_features=cat_features,
        eval_set=(X_val_sub, y_val_sub),
        early_stopping_rounds=50,
        use_best_model=True
    )

    # --- 5) Predict ---
    y_pred_val = model.predict(X_val_sub)

    # --- 6) Evaluate ---
    f1 = f1_score(y_val_sub, y_pred_val)

    # Return the F1 (Optuna will try to maximize this)
    return f1


In [89]:
# Create study that aims to maximize F1
import optuna
study = optuna.create_study(direction="maximize")

# Optimize over 'objective' for a certain number of trials
study.optimize(objective, n_trials=20, show_progress_bar=True)


[I 2025-01-23 21:27:34,790] A new study created in memory with name: no-name-8ce8d20c-b456-437c-8c24-278df2096bb7


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-01-23 21:27:52,779] Trial 0 finished with value: 0.14241381636645753 and parameters: {'iterations': 700, 'depth': 7, 'learning_rate': 0.09744541619010162, 'l2_leaf_reg': 4.6845889641307, 'bagging_temperature': 0.6083577649428933}. Best is trial 0 with value: 0.14241381636645753.
[I 2025-01-23 21:29:00,012] Trial 1 finished with value: 0.14715379312846932 and parameters: {'iterations': 900, 'depth': 8, 'learning_rate': 0.06056087732149749, 'l2_leaf_reg': 44.82689039760325, 'bagging_temperature': 0.3545388999625765}. Best is trial 1 with value: 0.14715379312846932.
[I 2025-01-23 21:29:18,804] Trial 2 finished with value: 0.14246593421096043 and parameters: {'iterations': 400, 'depth': 6, 'learning_rate': 0.012942977089435637, 'l2_leaf_reg': 23.56083041913574, 'bagging_temperature': 0.4558729858095575}. Best is trial 1 with value: 0.14715379312846932.
[I 2025-01-23 21:29:34,801] Trial 3 finished with value: 0.14265668849391955 and parameters: {'iterations': 700, 'depth': 7, 'learn

In [93]:
params = study.best_params

In [101]:
X_train.head()

,product,campaign_id,webpage_id,user_group_id,gender,age_level,user_depth,city_development_index,var_1,product_category
196,C,359520,13787,4,Male,4.0,3.0,NaN,0.0,4
197581,C,405490,60305,1,Male,1.0,3.0,2.0,0.0,3
43920,H,105960,11085,2,Male,2.0,3.0,4.0,0.0,5
332522,I,118601,28529,3,Male,3.0,3.0,2.0,1.0,4
45995,C,359520,13787,1,Male,1.0,3.0,3.0,0.0,4


In [439]:
# Import necessary libraries
import pandas as pd
import logging
from pathlib import Path
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold

# Define the ModelTrainer class
class ModelTrainer:
    def __init__(self, data_dir: False, model_name: str = "catboost", cat_features: list = None):
        #self.data_dir = Path(data_dir)
        self.model_name = model_name
        self.cat_features = cat_features

        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def load_data(self):
        self.logger.info(f"Loading preprocessed data from {self.data_dir}...")
        X_train = pd.read_pickle(self.data_dir / "X_train.pkl")
        y_train = pd.read_pickle(self.data_dir / "y_train.pkl").squeeze()
        return X_train, y_train

    def determine_categorical_features(self, X_train: pd.DataFrame):
        if self.cat_features:
            cat_features = [col for col in self.cat_features if col in X_train.columns]
        else:
            cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

        for col in cat_features:
            if col in X_train.columns:
                # Ensure column is treated as category
                X_train[col] = X_train[col].astype("category")
                
                # Add "missing" only if it's not already a category
                if "missing" not in X_train[col].cat.categories:
                    X_train[col] = X_train[col].cat.add_categories("missing")
                
                # Fill missing values with "missing"
                X_train[col] = X_train[col].fillna("missing")

        self.logger.info(f"Categorical features: {cat_features}")
        return cat_features


    def cross_validate_model(self, X_train: pd.DataFrame, y_train: pd.Series, cat_features: list, cv: int = 5):
        if self.model_name == 'catboost':
            model = CatBoostClassifier(
                random_seed=42, verbose=0, eval_metric='F1',
                cat_features=cat_features, class_weights=[1, 10]
            )
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

        self.logger.info(f"Performing {cv}-fold cross-validation...")
        skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

        fold_scores = []
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
            self.logger.info(f"Processing fold {fold + 1}...")

            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            self.logger.info(f"Validation set shape for fold {fold + 1}: {X_fold_val.shape}")

            model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_val, y_fold_val), use_best_model=True)

            fold_score = model.best_score_['validation']['F1']
            fold_scores.append(fold_score)

            self.logger.info(f"Fold {fold + 1} F1 score: {fold_score}")

        mean_cv_score = sum(fold_scores) / len(fold_scores)
        self.logger.info(f"Mean cross-validation F1 score: {mean_cv_score}")
        return mean_cv_score

    def train_model(self, X_train: pd.DataFrame, y_train: pd.Series, cat_features: list, val_size: float = 0.2):
        if self.model_name == 'catboost':
            model = CatBoostClassifier(
            random_seed=42,
            verbose=100,
            eval_metric='F1',
            cat_features=cat_features,
           #auto_class_weights='Balanced',
            max_depth=4,
           #colsample_bylevel=0.7,
            class_weights=[1, 1/a],
          # bagging_temperature=.4,
            grow_policy='SymmetricTree',
          # one_hot_max_size = 15,
           #earning_rate=0.2,
          
            bootstrap_type='Bernoulli',
            early_stopping_rounds=100,
            )

        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

        self.logger.info(f"Training {self.model_name} model...")
        X_train.drop(columns=['session_id', 'DateTime', 'user_id'], inplace=True, errors='ignore')

        X_train_final, X_valid, y_train_final, y_valid = train_test_split(
            X_train, y_train, test_size=val_size, random_state=42)
        

        self.logger.info(f"Training {self.model_name} model with validation set...")
        model.fit(X_train_final, y_train_final, eval_set=(X_valid, y_valid), use_best_model=True)

        return model

# Interactive Workflow for Jupyter Notebook
# Define the data directory
DATA_DIR = "path/to/data"  # Replace with your actual path

# Initialize ModelTrainer
trainer = ModelTrainer(DATA_DIR,model_name="catboost")

# Load data
#X_train, y_train = trainer.load_data()

# Determine categorical features
cat_features = trainer.determine_categorical_features(X_train)


INFO:__main__:Categorical features: ['product', 'campaign_id', 'webpage_id', 'gender', 'product_category']


In [436]:
X_train.head()

,product,campaign_id,webpage_id,user_group_id,gender,age_level,user_depth,city_development_index,var_1,product_category,hour,day,month
196,C,359520,13787,4,Male,4.0,3.0,NaN,0.0,4,11.0,4.0,7.0
197581,C,405490,60305,1,Male,1.0,3.0,2.0,0.0,3,7.0,6.0,7.0
43920,H,105960,11085,2,Male,2.0,3.0,4.0,0.0,5,13.0,4.0,7.0
332522,I,118601,28529,3,Male,3.0,3.0,2.0,1.0,4,22.0,7.0,7.0
45995,C,359520,13787,1,Male,1.0,3.0,3.0,0.0,4,12.0,2.0,7.0


In [437]:
X_train.drop('user_group_id', inplace=True, axis=1)
X_test.drop('user_group_id', inplace=True, axis=1)

In [440]:
model = trainer.train_model(X_train, y_train, cat_features)



INFO:__main__:Training catboost model...
INFO:__main__:Training catboost model with validation set...


Learning rate set to 0.121943
0:	learn: 0.5912225	test: 0.5956382	best: 0.5956382 (0)	total: 295ms	remaining: 4m 55s
100:	learn: 0.5904047	test: 0.5874867	best: 0.5959797 (1)	total: 41s	remaining: 6m 4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5959797396
bestIteration = 1

Shrink model to first 2 iterations.


In [431]:
X_test['hour'] = X_test['DateTime'].dt.hour
X_test['day'] = X_test['DateTime'].dt.day
X_test['month'] = X_test['DateTime'].dt.month

X_test = X_test.drop(columns=['session_id', 'DateTime', 'user_id'])

In [441]:
trainer.determine_categorical_features(X_test)

INFO:__main__:Categorical features: ['product', 'campaign_id', 'webpage_id', 'gender', 'product_category']


['product', 'campaign_id', 'webpage_id', 'gender', 'product_category']

In [442]:
y_pred = model.predict(X_test)

In [443]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
print(classification_report(y_test, y_pred,digits=4))

              precision    recall  f1-score   support

         0.0     0.9464    0.4739    0.6316     68166
         1.0     0.0811    0.6334    0.1437      4994

    accuracy                         0.4848     73160
   macro avg     0.5137    0.5536    0.3876     73160
weighted avg     0.8873    0.4848    0.5983     73160



In [411]:
import numpy as np
from sklearn.metrics import f1_score

# After training your CatBoost model:
y_probs = model.predict_proba(X_test)[:, 1]

best_threshold = 0.0
best_f1 = 0.0

# We can search thresholds from 0.0 to 1.0 in small steps
for t in np.linspace(0, 1, 101):
    y_pred = (y_probs >= t).astype(int)
    score = f1_score(y_test, y_pred)
    if score > best_f1:
        best_f1 = score
        best_threshold = t

print(f"Best threshold: {best_threshold}")
print(f"Best F1 score: {best_f1}")

# When predicting on test data, use the best_threshold:
y_probs_test = model.predict_proba(X_test)[:, 1]
y_pred_test = (y_probs_test >= best_threshold).astype(int)


Best threshold: 0.53
Best F1 score: 0.15259781911481718


## Naive preprocessing (not my class)

In [17]:
data2= raw_data.drop(columns = ['session_id', 'DateTime', 'user_id','product_category_2'])
data2.dropna(inplace = True) # Just close your eyes and drop the rows with missing values

In [18]:
X_2 = data2.drop(columns = ['is_click'])
y_2 = data2['is_click']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

In [19]:
columns_to_d2 = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "product_category_1"]

X_train_2_d = pd.get_dummies(X_train_2, columns = columns_to_d2)
X_test_2_d = pd.get_dummies(X_test_2, columns = columns_to_d2)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_2_d, y_train_2)
y_pred_RF = rf.predict(X_test_2_d)



In [21]:
f1_score(y_test_2, y_pred_RF)

0.14298031865042174

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_2_d, y_train_2)

y_pred_LR = model.predict(X_test_2_d)
print(f1_score(y_test_2, y_pred_LR))

0.14430736693690518
